<a href="https://colab.research.google.com/github/ufrpe-mineracao-textos/projeto-de-mineracao-20192-equipy/blob/master/equiPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cloning the repository**

In [5]:
!git clone https://github.com/ufrpe-mineracao-textos/projeto-de-mineracao-20192-equipy.git

fatal: destination path 'projeto-de-mineracao-20192-equipy' already exists and is not an empty directory.


In [6]:
import spacy
import pandas as pd

!python -m spacy download pt

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [15]:


nlp = spacy.load('pt')

path = "projeto-de-mineracao-20192-equipy/"
data = pd.read_csv(path+"new_dataset.csv")



doc=[]

for contador in range(2001):
  doc.append(nlp(str(data["text"][contador])))


#print(doc[1])

a=[]
c=0
#for c in range(2):
#  a.append([doc.text.lower() for doc in doc if not(doc.is_stop or doc.is_punct)])

#print(a)
b=[]
for k in range(2001):
  for token in doc[k]:
    if not (token.is_stop or token.is_punct):
      b.append(token.text.lower())
  a.append([b])
  b=[]
print(a)
#print(data["text"])#[0])


texto, classe = data["text"], data["label"]
#texto[0]
#data

[[['sábado', 'o', 'sala', 'falar', 'matéria', 'haja', 'saúde', 'mental']], [['o', 'tema', 'amore', 'k', '12', 'fala', 'rivalidade', 'feminina', 'bulimia', 'traição', 'pedófilia', 'assédio', 'padrão', 'beleza', 'indústria', 'música', 'pressão', 'lançar', 'álbuns', 'saúde', 'mental', 'coisas', 'cry', 'baby', 'fala', 'relacionamento', 'abusivo', 'bullying', 'família']], [['queria', 'ninha', 'saúde', 'mental', 'volta']], [['saude', 'mental', 'ruim', 'p', 'pior']], [['a', 'saúde', 'mental', 'd', 'afeta', 'mundo', 'casa', 'e', 'ninguém', 'o', 'direto', 'surtar', 'chorar', 'paz', 'pra', 'escola', 'pra', 'vó', 'pq', 'únicos', 'momentos', 'q', 'fico', 'casa']], [['setembro', 'amarelo', 'finanças', 'afetam', 'a', 'saúde', 'mental']], [['saúde', 'mental', 'ruim', 'jogaria', 'd']], [['custei', 'p', 'entender', 'e', 'aceitar', 'q', 'a', 'gnt', 'e', 'pessoas', 'o', 'contrário', 'd', 'q', 'custou', 'a', 'mnh', 'paz', 'mnh', 'saúde', 'mental', 'e', 'msm', 'a', 'mnh', 'felicidade', 'agradeço', 'a', 'de

In [0]:
#Espaco pro Stemming/Lemming

In [0]:
#Espaco pro Stemming/Lemming

In [0]:
treino_texto = texto[:1300]
teste_texto = texto[1301:2000]
treino_classe = classe[:1300]
teste_classe = classe[1301:2000]

# Extração de caracteristicas

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(encoding='latin-1')
X_treino_counts = count_vect.fit_transform(treino_texto)
X_treino_counts.shape

(1300, 4137)

In [0]:
count_vect.vocabulary_.get('triste') #so pra imprimir quantas palavras 'triste' se encontra nos tweets

3874

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=True)#decide se vai usar o TF ou TF-IDF
X_train_tfidf = tfidf_transformer.fit_transform(X_treino_counts)
X_train_tfidf.shape

(1300, 4137)